In [1]:
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

In [2]:
# --- 1. CARREGAR O NOSSO MELHOR MODELO (v2 do Active Learning) ---
model_path = "/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_v2_final"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [3]:
# --- 2. CARREGAR TODOS OS DADOS NÃO ROTULADOS ---
# Carregar o dataset completo e os IDs já anotados manualmente
df_full = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/interim/simulados_enem_completo.csv')
df_annotated = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv') # Usando o seed set como base de já anotados

# Filtrar para obter apenas os dados que AINDA NÃO foram anotados
unlabeled_df = df_full[~df_full['question_id'].isin(df_annotated['question_id'])].copy()
unlabeled_df = unlabeled_df.rename(columns={'question': 'text'})

print(f"Encontrados {len(unlabeled_df)} exemplos não rotulados para a Pseudo-Rotulagem.")
unlabeled_dataset = Dataset.from_pandas(unlabeled_df)

Encontrados 7157 exemplos não rotulados para a Pseudo-Rotulagem.


In [4]:
# --- 3. FAZER PREDIÇÕES E FILTRAR POR CONFIANÇA ---
# Tokenizar os dados não rotulados
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_unlabeled_dataset = unlabeled_dataset.map(tokenize_function, batched=True)

# Criar um 'Trainer' apenas para predições
trainer = Trainer(model=model)

# Obter as predições (logits)
predictions = trainer.predict(tokenized_unlabeled_dataset)
logits = predictions.predictions

# Converter logits para probabilidades
probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1).numpy()

# Obter a confiança (probabilidade máxima) e a classe prevista para cada exemplo
confidence = np.max(probabilities, axis=1)
predicted_class_ids = np.argmax(probabilities, axis=1)

# Adicionar essa informação ao nosso dataframe
unlabeled_df['pseudo_label_id'] = predicted_class_ids
unlabeled_df['confidence'] = confidence

# Definir o nosso ponto de corte de confiança
CONFIDENCE_THRESHOLD = 0.90

# Filtrar o dataframe para manter apenas as predições de alta confiança
df_pseudo_labeled_train = unlabeled_df[unlabeled_df['confidence'] > CONFIDENCE_THRESHOLD].copy()

# Mapear os IDs de volta para os nomes das classes para análise
id2label = model.config.id2label
df_pseudo_labeled_train['pseudo_label'] = df_pseudo_labeled_train['pseudo_label_id'].map(id2label)

Map:   0%|          | 0/7157 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [5]:
# --- 4. ANALISAR E SALVAR O NOVO DATASET DE TREINO ---
print(f"\nConseguimos gerar {len(df_pseudo_labeled_train)} pseudo-rótulos com confiança > {CONFIDENCE_THRESHOLD*100}%.")
print("\nDistribuição dos pseudo-rótulos:")
print(df_pseudo_labeled_train['pseudo_label'].value_counts())

# Salvar este dataset para a próxima etapa de treinamento
df_pseudo_labeled_train.to_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/pseudo_labeled_train_set.csv', index=False)
print("\nArquivo com o novo dataset de treino salvo em '/Users/giossaurus/Developer/leia_tcc/data/processed/pseudo_labeled_train_set.csv'")


Conseguimos gerar 0 pseudo-rótulos com confiança > 90.0%.

Distribuição dos pseudo-rótulos:
Series([], Name: count, dtype: int64)

Arquivo com o novo dataset de treino salvo em '/Users/giossaurus/Developer/leia_tcc/data/processed/pseudo_labeled_train_set.csv'
